In [4]:
#이름이랑 별점만 크롤링
import json
import time
from time import sleep
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

def search_food(key):
    
    cafe_list = []
    
    url = 'https://map.naver.com/v5/search'
    driver = webdriver.Chrome()  # 드라이버
    driver.get(url)
    key_word = "서울 " + key + ' 카페'  # 검색어

    # css 찾을때 까지 10초대기
    def time_wait(num, code):
        try:
            wait = WebDriverWait(driver, num).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, code)))
        except:
            print(code, '태그를 찾지 못하였습니다.')
            driver.quit()
        return wait

    # frame 변경 메소드
    def switch_frame(frame):
        driver.switch_to.default_content()  # frame 초기화
        driver.switch_to.frame(frame)  # frame 변경

    # 페이지 다운
    def page_down(num):
        body = driver.find_element(By.CSS_SELECTOR, 'body')
        body.click()
        for i in range(num):
            body.send_keys(Keys.PAGE_DOWN)

    # css를 찾을때 까지 10초 대기
    time_wait(10, 'div.input_box > input.input_search')

    # (1) 검색창 찾기
    search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
    search.send_keys(key_word)  # 검색어 입력
    search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

    sleep(1)

    # (2) frame 변경
    switch_frame('searchIframe')
    page_down(40)
    sleep(3)

    # 가게 리스트
    shop_list = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')
    # 페이지 리스트
    next_btn = driver.find_elements(By.CSS_SELECTOR, '.zRM9F> a')

    # dictionary 생성
    shop_dict = {'가게정보': []}
    # 시작시간
    start = time.time()
    print('[크롤링 시작...]')

    # 크롤링 (페이지 리스트 만큼)
    for btn in range(len(next_btn))[1:]:  # next_btn[0] = 이전 페이지 버튼 무시 -> [1]부터 시작
        parking_list = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')
        names = driver.find_elements(By.CSS_SELECTOR, '.TYaxT')  # (3) 장소명
        types = driver.find_elements(By.CSS_SELECTOR, '.KCMnt')  # (4) 장소 유형
        star = driver.find_elements(By.CSS_SELECTOR, '.h69bs.orXYY') #별점 박스

        for data in range(len(shop_list)):  # 가게 리스트 만큼
            print(data)
            #테스트 부분
            print(star[data])
            real_star = star[data].text
            
            real_star = str(real_star).split("\n")[1]
            #print(real_star)
            if float(real_star) < 4.3:
                continue

            sleep(1)
            try:

                # (3) 가게명 가져오기
                shop_name = str(names[data].text)
                print(shop_name)

                # (4) 유형
                shop_type = str(types[data].text)
                print(shop_type)
                
                #(5) 별점
                real_star = star[data].text
                real_star = str(real_star).split("\n")[1]
                print(real_star)
                
                tmp = [shop_name, real_star]
                cafe_list.append(tmp)
                
                print(f'{shop_name} ...완료')

                sleep(1)

            except Exception as e:
                print(e)
                print('ERROR!' * 3)

                sleep(1)

        # 다음 페이지 버튼 누를 수 없으면 종료
        if not next_btn[-1].is_enabled():
            break

        if names[-1]:  # 마지막이면 다음버튼 클릭
            next_btn[-1].click()

            sleep(2)

        else:
            print('페이지 인식 못함')
            break

    print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
    driver.quit()  # 작업이 끝나면 창을 닫는다.

    index_final = len(cafe_list) // 2
    cafe_list = cafe_list[0:index_final]
    
    return cafe_list

##############################################################################

regions = ["종로구", "중구", "용산구", "성동구", "광진구", "동대문구", "중랑구", "성북구", "강북구", "도봉구", "노원구", "은평구", "서대문구", "마포구", "양천구", 
           "강서구", "구로구", "금천구", "영등포구", "동작구", "관악구", "서초구", "강남구", "송파구", "강동구"]


for i in range(1, 2):
    filename = "./out_cafe_new/out_" + str(i) + "_cafe.csv"
    f = open(filename, 'w', encoding='utf-8', newline='')
    writer = csv.writer(f)
    writer.writerows(search_food(regions[i]))
    f.close()

The chromedriver version (128.0.6613.137) detected in PATH at c:\Users\kk21\Desktop\TRIPER\chromedriver.exe might not be compatible with the detected chrome version (129.0.6668.90); currently, chromedriver 129.0.6668.91 is recommended for chrome 129.*, so it is advised to delete the driver in PATH and retry


[크롤링 시작...]
0
커피그래
카페,디저트
4.56
커피그래 ...완료
1
콘티뉴이티
카페,디저트
4.42
콘티뉴이티 ...완료
2
덕수궁 리에제와플
와플
4.36
덕수궁 리에제와플 ...완료
3
더 스팟 패뷸러스
카페,디저트
4.33
더 스팟 패뷸러스 ...완료
4
커피한약방
카페
4.47
커피한약방 ...완료
5
르풀
카페
4.44
르풀 ...완료
6
호텔수선화
카페
4.43
호텔수선화 ...완료
7
맷차 명동본점
카페,디저트
4.44
맷차 명동본점 ...완료
8
커피루소 정동
카페,디저트
4.63
커피루소 정동 ...완료
9
라운드앤드
베이커리
4.47
라운드앤드 ...완료
10
가배도 명동
카페
4.45
가배도 명동 ...완료
11
스몰굿커피 플래그십스토어
카페
4.38
스몰굿커피 플래그십스토어 ...완료
12
13
알렉스룸
카페,디저트
4.71
알렉스룸 ...완료
14
르빵 명동성당점
베이커리
4.63
르빵 명동성당점 ...완료
15
적당
카페,디저트
4.55
적당 ...완료
16
태극당
베이커리
4.43
태극당 ...완료
17
분카샤 을지로점
카페,디저트
4.43
분카샤 을지로점 ...완료
18
공간갑
카페,디저트
4.61
공간갑 ...완료
19
리사르커피 약수점
카페,디저트
4.44
리사르커피 약수점 ...완료
20
빠우 을지로점
도넛
4.51
빠우 을지로점 ...완료
21
커피앤시가렛
카페
4.75
커피앤시가렛 ...완료
22
스테츄
카페,디저트
4.38
스테츄 ...완료
23
쌀베베 서울시청역점
베이글
4.38
쌀베베 서울시청역점 ...완료
24
호랑이
카페
4.51
호랑이 ...완료
25
수잔나의 앞치마
카페,디저트
4.59
수잔나의 앞치마 ...완료
26
파오리
카페
4.32
파오리 ...완료
27
커피그래
카페,디저트
4.5
커피그래 ...완료
28
더하우스1932
카페,디저트
4.86
더하우스1932 ...완료
29
브릭하우스
카페,디저트
4.52
브릭하우스 ...완료
30
31
섬광
카페,디저트
4.61
섬광 ...완료
32
코인 

IndexError: list index out of range